# Black FRIDAY SALES Prediction 
Model explanination using eli5, Lime, shape 

In [ ]:
#Import all lib
import pandas as pd
from sklearn import * 

In [ ]:
train_url="../input/black-friday-sales-prediction/train_oSwQCTC (1)/train.csv"
test_url="../input/black-friday-sales-prediction/test_HujdGe7 (1)/test.csv"

df=pd.read_csv(train_url)

print('---------------------')
print('No. of rows: {} and columns: {}'.format(df.shape[0],df.shape[1]))
print('----------------------')
print('Cloumns names: {}'.format(df.columns))
print('\n----------------------')
print(df.head())

*Features information*
* User_ID: Unique identifier of shopper.
* Product_ID: Unique identifier of product. (No key given)
* Gender: Sex of shopper.
* Age: Age of shopper split into bins.
* Occupation: Occupation of shopper. (No key given)
* City_Category: Residence location of shopper. (No key given)
* Stay_In_Current_City_Years: Number of years stay in current city.
* Marital_Status: Marital status of shopper.
* Product_Category_1: Product category of purchase.
* Product_Category_2: Product may belong to other category.
* Product_Category_3: Product may belong to other category.
* Purchase: Purchase amount in dollars.

In [ ]:
print('----------Data Types--------------')
print(df.info())
print('\n-------Total Null values in each column---------')
print(df.isnull().sum())

In [ ]:
df.drop(columns=['Product_ID','User_ID','Product_Category_2','Product_Category_3',],inplace=True)

print('---------------------')
print('No. of rows: {} and columns: {}'.format(df.shape[0],df.shape[1]))

In [ ]:
num_features=[]
cat_features=[]
for i in df.columns:
    if df[i].dtype=="object":
        cat_features.append(i)
    else:
        num_features.append(i)
num_features.remove('Purchase')
target = ['Purchase']
features=num_features+cat_features
print('--------num_features---------')
print(num_features)
print('--------cat_features---------')
print(cat_features)
print('--------Features---------')
print(features)
print('---------Target----------')
print(target)

## Building the model and pipeline

In [ ]:
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(df[features], 
                                                                     df['Purchase'],
                                                                     test_size=.3, 
                                                                     random_state=42)

In [ ]:
# Preprocessing for categorical data
categorical_transformer = preprocessing.OneHotEncoder(handle_unknown='ignore')


num_transformer=preprocessing.MinMaxScaler()
# Bundle preprocessing for numerical and categorical data
preprocessor = compose.ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

In [ ]:
# model = ensemble.RandomForestRegressor()
model = linear_model.Lasso()

clf=pipeline.Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)


In [ ]:
# Evaluate the model
import numpy as np
score = np.sqrt(metrics.mean_squared_error(y_valid, preds))
print('MAE:', score)

In [ ]:
import eli5

#To extract the features from OneHotEncoding
preprocessor = clf.named_steps["preprocessor"]
ohe_categories = preprocessor.named_transformers_["cat"].categories_
new_ohe_features = [f"{col}__{val}" for col, vals in zip(cat_features, ohe_categories) for val in vals]

#All feature column
all_features = num_features + new_ohe_features

#Show weights
eli5.show_weights(clf.named_steps["model"], feature_names=all_features)